In [1]:
import ee
import geemap
import pandas as pd

In [2]:
Map = geemap.Map()

In [3]:
df = pd.read_excel("comunas.xlsx")
comunas = df["COD_COM"].unique().tolist()

In [4]:
def registros(cod, cant):
    diccionario = {}
    diccionario["COMUNA"] = cod
    diccionario[year] = cant

    return diccionario

In [5]:
# startDate = ['2000-01-01', '2001-01-01', '2002-01-01', '2003-01-01', '2004-01-01', '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01']
# endDate = ['2001-01-01', '2002-01-01', '2003-01-01', '2004-01-01', '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']

startDate = ['2004-01-01', '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01']
endDate = ['2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']

In [10]:
for i in zip(startDate,endDate):
    
    Fecha_inicial = str(i[0])
    Fecha_final = str(i[1])
    
    year = Fecha_inicial[:4]
    
    puntos = []

    for i in comunas:
        studyArea = geemap.shp_to_ee("data/comunas/Lim_comunas.shp")
        studyArea = studyArea.filterMetadata("COMUNA","equals", str(i))

        FIRMS_colection2 = ee.ImageCollection('FIRMS')
        FIRMS_colection = ee.ImageCollection('FIRMS')

        FIRMS4 =FIRMS_colection2 \
          .select(['T21']) \
          .filterDate(Fecha_inicial,Fecha_final) \
          .filterBounds(studyArea)

        FIRMS =FIRMS_colection \
          .select(['T21']) \
          .filterDate(Fecha_inicial,Fecha_final) \
          .filterBounds(studyArea)

        FIRMScount4  = ee.Image(FIRMS4.count()).clip(studyArea)
        FIRMSbinary4 = FIRMScount4.eq(FIRMScount4).rename('FIRMS_binary_alert_3')

        project_crs   = ee.Image(FIRMS.first()).projection().crs()
        scale = ee.Image(FIRMS.first()).projection().nominalScale()

        FIRMSpoint4 = FIRMSbinary4.reduceToVectors(
          geometry = studyArea,
          eightConnected=True,
          labelProperty='modis_fire',
          maxPixels=1e16,
          crs=project_crs,
          scale=scale,
          geometryType= 'centroid',
          bestEffort= True,
          tileScale= 16
        )

        numero_PI = ee.FeatureCollection(FIRMSpoint4).filterBounds(studyArea)

        cantidad_PI = numero_PI.size()
        cantidad =  cantidad_PI.getInfo()

        diccionario = registros(i, cantidad)
        puntos.append(diccionario.copy())

    data = pd.DataFrame(puntos)
    data.to_excel(str(year) + ".xlsx", index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Hector\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-54b945a6e3da>", line 48, in <module>
    cantidad =  cantidad_PI.getInfo()
  File "C:\Users\Hector\anaconda3\lib\site-packages\ee\computedobject.py", line 98, in getInfo
    return data.computeValue(self)
  File "C:\Users\Hector\anaconda3\lib\site-packages\ee\data.py", line 672, in computeValue
    return _execute_cloud_call(
  File "C:\Users\Hector\anaconda3\lib\site-packages\ee\data.py", line 334, in _execute_cloud_call
    return call.execute(num_retries=num_retries)
  File "C:\Users\Hector\anaconda3\lib\site-packages\googleapiclient\_helpers.py", line 134, in positional_wrapper
    return wrapped(*args, **kwargs)
  File "C:\Users\Hector\anaconda3\lib\site-packages\googleapiclient\http.py", line 900, in execute
    resp, content = _retry_request(
  File "C:\User

TypeError: object of type 'NoneType' has no len()